In [1]:
import polars as pl
from src.tank_ts_endpoint import fetch_tank_ts_data, GetTanksTimeDataReq

req = GetTanksTimeDataReq(scada_id='9882701')

df = await fetch_tank_ts_data(req)
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

ts,value,scada_id,tank_name,uom
datetime[μs],f64,str,str,str
2023-12-16 00:14:23.709805,129.191933,"""9882701""","""OilTank1Volume""","""bbl"""
2023-10-20 06:11:00.976191,242.917252,"""9882701""","""OilTank1Volume""","""bbl"""
2019-02-17 18:24:15.392353,467.167758,"""9882701""","""OilTank1Volume""","""bbl"""
2024-01-06 21:34:39.673218,69.010569,"""9882701""","""OilTank1Level""","""in"""
2024-07-20 01:43:20.665138,93.046905,"""9882701""","""OilTank1Level""","""in"""
2017-04-13 01:06:18.038402,87.370372,"""9882701""","""OilTank1Level""","""in"""


In [2]:
lf = lf.drop("key_metric")
lf.collect()

ts,value,scada_id,tank_name,uom
datetime[μs],f64,str,str,str
2023-12-16 00:14:23.709805,129.191933,"""9882701""","""OilTank1Volume""","""bbl"""
2023-10-20 06:11:00.976191,242.917252,"""9882701""","""OilTank1Volume""","""bbl"""
2019-02-17 18:24:15.392353,467.167758,"""9882701""","""OilTank1Volume""","""bbl"""
2024-01-06 21:34:39.673218,69.010569,"""9882701""","""OilTank1Level""","""in"""
2024-07-20 01:43:20.665138,93.046905,"""9882701""","""OilTank1Level""","""in"""
2017-04-13 01:06:18.038402,87.370372,"""9882701""","""OilTank1Level""","""in"""


In [3]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_esd>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("tank_name").str.extract_groups(pattern))
lf = lf.unnest("separated_metrics")
lf.collect()

ts,value,scada_id,tank_name,uom,is_esd,tank_type,tank_number,tank_metric
datetime[μs],f64,str,str,str,str,str,str,str
2023-12-16 00:14:23.709805,129.191933,"""9882701""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
2023-10-20 06:11:00.976191,242.917252,"""9882701""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
2019-02-17 18:24:15.392353,467.167758,"""9882701""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
2024-01-06 21:34:39.673218,69.010569,"""9882701""","""OilTank1Level""","""in""",null,"""Oil""","""1""","""Level"""
2024-07-20 01:43:20.665138,93.046905,"""9882701""","""OilTank1Level""","""in""",null,"""Oil""","""1""","""Level"""
2017-04-13 01:06:18.038402,87.370372,"""9882701""","""OilTank1Level""","""in""",null,"""Oil""","""1""","""Level"""


In [4]:
lf = lf.drop("tank_name", "is_esd", "tank_type", "tank_number")
lf.collect()

ts,value,scada_id,uom,tank_metric
datetime[μs],f64,str,str,str
2023-12-16 00:14:23.709805,129.191933,"""9882701""","""bbl""","""Volume"""
2023-10-20 06:11:00.976191,242.917252,"""9882701""","""bbl""","""Volume"""
2019-02-17 18:24:15.392353,467.167758,"""9882701""","""bbl""","""Volume"""
2024-01-06 21:34:39.673218,69.010569,"""9882701""","""in""","""Level"""
2024-07-20 01:43:20.665138,93.046905,"""9882701""","""in""","""Level"""
2017-04-13 01:06:18.038402,87.370372,"""9882701""","""in""","""Level"""


In [5]:
aggregated_lf = lf.group_by(["tank_metric", "uom"]).agg([
    pl.col("ts").alias("timestamps"),
    pl.col("value").alias("values")
])

aggregated_lf.collect()

tank_metric,uom,timestamps,values
str,str,list[datetime[μs]],list[f64]
"""Volume""","""bbl""","[2023-12-16 00:14:23.709805, 2023-10-20 06:11:00.976191, 2019-02-17 18:24:15.392353]","[129.191933, 242.917252, 467.167758]"
"""Level""","""in""","[2024-01-06 21:34:39.673218, 2024-07-20 01:43:20.665138, 2017-04-13 01:06:18.038402]","[69.010569, 93.046905, 87.370372]"
